In [1]:
import duckdb
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import polars as pl
from pandas.api.types import infer_dtype

In [2]:
# Using duckdb query to avoid "ValueError" from pd.read_json() from id columns
def duck_json(file_str):
    df = duckdb.query(f'''SELECT * FROM read_json({file_str}, auto_detect=True, sample_size=100000)''').to_df() 
    df.name = file_str
    return df


shots = duck_json('shots.json')
games = duck_json('games.json')
points = duck_json('points.json')
matches = duck_json('matches.json')

df_list = [shots, games, points, matches]

In [3]:
shapes = {}
for df in df_list:
    shapes[df.name] = df.shape

shapes

{'shots.json': (62249, 20),
 'games.json': (2128, 12),
 'points.json': (11732, 17),
 'matches.json': (100, 17)}

In [4]:
def infertypes(df):
    df_dtypes = {}
    for col in df.columns:
        df_dtypes[col] = infer_dtype(df[col])
    return df_dtypes

# Finds unique values of all columns, raises error if there are unhashable values in a column
def find_unique(df):
    dtype_dic = infertypes(df)
    for column_name in dtype_dic.keys():
        assert dtype_dic.get(column_name) != 'mixed', column_name + ' has mixed or unhashable values'
    unique_dic = {}
    for col in df.columns:
        unique_dic[col] = pd.unique(df[col])
    return unique_dic


In [5]:
shots_dtypes = infertypes(shots)
shots_dtypes

{'match_id': 'floating',
 'user_id': 'floating',
 'sid': 'integer',
 'pid': 'integer',
 'set_id': 'integer',
 'game_id': 'integer',
 'player': 'string',
 'shot_type': 'string',
 'hit_court_side': 'string',
 'hit_type': 'string',
 'hit_location_long': 'string',
 'hit_location_lat': 'string',
 'hit_location': 'mixed',
 'hit_velocity': 'mixed',
 'hit_wing': 'string',
 'net_type': 'string',
 'bounce_court_side': 'string',
 'bounce_location_long': 'string',
 'bounce_location_lat': 'string',
 'bounce_location': 'mixed'}

In [6]:
shots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62249 entries, 0 to 62248
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   match_id              62249 non-null  float64
 1   user_id               52796 non-null  float64
 2   sid                   62249 non-null  int64  
 3   pid                   62249 non-null  int64  
 4   set_id                62249 non-null  int64  
 5   game_id               62249 non-null  int64  
 6   player                62249 non-null  object 
 7   shot_type             62249 non-null  object 
 8   hit_court_side        62249 non-null  object 
 9   hit_type              62249 non-null  object 
 10  hit_location_long     62249 non-null  object 
 11  hit_location_lat      62249 non-null  object 
 12  hit_location          62249 non-null  object 
 13  hit_velocity          62249 non-null  object 
 14  hit_wing              62249 non-null  object 
 15  net_type           

In [7]:
# Get rid of unhashable columns for dictionary, typically columns with lists as values
nolist_shots = shots.copy(deep = True)
nolist_shots = nolist_shots.drop(['hit_location', 'hit_velocity', 'bounce_location'], axis = 'columns')

shots_unique_vals = find_unique(nolist_shots)
shots_unique_vals

{'match_id': array([9.64239483e+47, 1.40705126e+48, 2.70115592e+47, 1.29562332e+48,
        5.26074506e+47, 3.23485238e+47, 9.00750802e+47, 2.45769257e+47,
        8.47861948e+47, 8.86922998e+47, 2.38088724e+47, 9.33140447e+46,
        1.08648182e+48, 5.31353632e+47, 7.21253803e+47, 1.04062547e+48,
        6.06385600e+47, 8.27383837e+47, 6.15434493e+47, 8.93286255e+45,
        3.41374592e+47, 1.18943352e+48, 3.45165938e+47, 1.18834967e+48,
        1.02639901e+48, 1.41251282e+48, 9.24518323e+47, 4.16780906e+47,
        4.13281520e+47, 1.13366570e+48, 1.26188679e+48, 9.37193324e+47,
        6.94698592e+47, 4.80016873e+47, 4.11120413e+47, 9.60882826e+47,
        9.24521909e+47, 7.26846270e+47, 1.04266754e+48, 2.69780713e+47,
        7.52182052e+46, 5.58556317e+47, 1.44765392e+48, 1.59578755e+47,
        6.69112456e+47, 3.67155304e+47, 2.12065335e+47, 1.29137449e+48,
        6.78517162e+47, 3.98888137e+47, 1.27391318e+48, 2.66504147e+47,
        2.18345738e+47, 8.75823036e+47, 6.70885915e+

In [8]:
# empty dataframe shows there's no shots that have 'net' and bounce over to other side, so this column is probably just to show if the shot hits net and doesn't go over
shots[(shots['net_type'] == 'net' ) & (shots['hit_court_side'] == 'near') & (shots['bounce_court_side'] == 'far')]

,match_id,user_id,sid,pid,set_id,game_id,player,shot_type,hit_court_side,hit_type,hit_location_long,hit_location_lat,hit_location,hit_velocity,hit_wing,net_type,bounce_court_side,bounce_location_long,bounce_location_lat,bounce_location


In [9]:
points[points['is_rally'] == False]

,pid,match_id,game_id,set_id,did_host_win_point,detail,was_break_point,was_set_point_for_host,was_set_point_for_guest,num_of_serves,server,prev_host_points,prev_guest_points,is_rally,started_at,ended_at,rally_length
56,57,9.642395e+47,1,1,True,double_fault,False,False,False,2,guest,0,0,False,2023-05-29T11:30:54.076000Z,2023-05-29T11:31:00.080000Z,1
57,58,9.642395e+47,1,1,False,service_winner,False,False,False,2,guest,1,0,False,2023-05-29T11:36:24.478000Z,2023-05-29T11:36:25.379000Z,1
58,59,9.642395e+47,1,1,True,forehand_winner,False,False,False,1,guest,1,1,False,2023-05-29T11:36:53.096000Z,2023-05-29T11:36:54.213000Z,1
59,60,9.642395e+47,1,1,False,backhand_unforced_error,False,False,False,2,guest,2,1,False,2023-05-29T11:37:31.619000Z,2023-05-29T11:37:37.723000Z,3
60,61,9.642395e+47,1,1,True,backhand_unforced_error,False,False,False,1,guest,2,2,False,2023-05-29T11:38:24.652000Z,2023-05-29T11:38:32.857000Z,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11727,82,3.666105e+46,14,2,False,normal,False,False,False,1,host,1,0,False,2023-05-26T14:33:03.179000Z,2023-05-26T14:33:07.548000Z,3
11728,83,3.666105e+46,14,2,False,normal,False,False,False,1,host,1,1,False,2023-05-26T14:33:29.323000Z,2023-05-26T14:33:33.709000Z,3
11729,84,3.666105e+46,14,2,True,normal,False,False,False,1,host,1,2,False,2023-05-26T14:33:56.768000Z,2023-05-26T14:34:02.721000Z,4
11730,85,3.666105e+46,14,2,True,normal,False,False,False,1,host,2,2,False,2023-05-26T14:34:20.178000Z,2023-05-26T14:34:27.531000Z,4


In [10]:
points_dtypes = infertypes(points)
points_dtypes

{'pid': 'integer',
 'match_id': 'floating',
 'game_id': 'integer',
 'set_id': 'integer',
 'did_host_win_point': 'boolean',
 'detail': 'string',
 'was_break_point': 'boolean',
 'was_set_point_for_host': 'boolean',
 'was_set_point_for_guest': 'boolean',
 'num_of_serves': 'integer',
 'server': 'string',
 'prev_host_points': 'integer',
 'prev_guest_points': 'integer',
 'is_rally': 'boolean',
 'started_at': 'string',
 'ended_at': 'string',
 'rally_length': 'integer'}

In [11]:
points_unique = find_unique(points)
points_unique

{'pid': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 1

In [12]:
match_types = infertypes(matches)
match_types

{'id': 'floating',
 'host_id': 'floating',
 'guest_id': 'floating',
 'auto_scored': 'boolean',
 'sets_per_match': 'integer',
 'games_per_set': 'integer',
 'is_ad': 'boolean',
 'is_tiebreak': 'boolean',
 'is_super_tiebreak': 'boolean',
 'score_tier': 'string',
 'original_score_tier': 'string',
 'winner': 'string',
 'total_points': 'integer',
 'host_shot_count': 'integer',
 'guest_shot_count': 'integer',
 'host_right_handed': 'boolean',
 'guest_right_handed': 'boolean'}

In [13]:
match_unique = find_unique(matches)
match_unique

{'id': array([9.64239483e+47, 1.40705126e+48, 5.31963862e+47, 2.70115592e+47,
        1.29562332e+48, 5.26074506e+47, 3.23485238e+47, 9.00750802e+47,
        2.45769257e+47, 8.47861948e+47, 8.86922998e+47, 2.38088724e+47,
        9.33140447e+46, 1.08648182e+48, 5.31353632e+47, 7.21253803e+47,
        1.04062547e+48, 6.06385600e+47, 8.27383837e+47, 6.15434493e+47,
        8.93286255e+45, 3.41374592e+47, 1.18943352e+48, 3.45165938e+47,
        1.18834967e+48, 1.02639901e+48, 1.41251282e+48, 9.24518323e+47,
        4.16780906e+47, 1.41077155e+48, 4.13281520e+47, 1.13366570e+48,
        1.26188679e+48, 9.37193324e+47, 6.94698592e+47, 4.80016873e+47,
        4.11120413e+47, 9.60882826e+47, 9.24521909e+47, 7.26846270e+47,
        1.04266754e+48, 2.69780713e+47, 7.52182052e+46, 5.58556317e+47,
        1.44765392e+48, 1.59578755e+47, 6.69112456e+47, 3.67155304e+47,
        2.12065335e+47, 1.29137449e+48, 6.78517162e+47, 3.98888137e+47,
        1.27391318e+48, 2.66504147e+47, 2.18345738e+47, 8.